In [1]:
import pandas as pd
from question_answering.utils import core_qa_utils, generative_qa_utils
from question_answering.paths import generative_qa_paths
from transformers import (
    BartTokenizerFast,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    TFAutoModelForSeq2SeqLM,
)
import os
from tqdm import tqdm
import numpy as np

In [2]:
java_samples_path = generative_qa_paths.code_qa_dataset_dir / 'java' / "manual_test.csv"
python_samples_path = generative_qa_paths.code_qa_dataset_dir / 'java' / "manual_test.csv"

java_samples = pd.read_csv(java_samples_path)
python_samples = pd.read_csv(python_samples_path)

In [3]:
java_dataset, python_dataset = core_qa_utils.convert_dataframes_to_datasets(
    [java_samples, python_samples]
)

In [4]:
model_names = os.listdir(generative_qa_paths.saved_models_dir)
batch_size = 8
prediction_dataframe_list = []
prediction_dataframe_dict = {}
index_to_start_from = 11

i = 0
for model_name in tqdm(model_names):
    if i < index_to_start_from:
        i = i + 1
        continue
    if "bart" in model_name:
        model_checkpoint = "facebook/bart-base"
        tokenizer = BartTokenizerFast.from_pretrained(model_checkpoint)
        model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, from_pt=True)
        data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")
        max_length = 256
    elif "t5" in model_name:
        model_checkpoint = "t5-base"
        tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
        model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, from_pt=True)
        data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")
        max_length = 512

    if "original" in model_name:
        code_type = 'original_code'
    elif "normalised" in model_name:
        code_type = 'code'

    if "java" in model_name:
        dataset = java_dataset
    elif "python" in model_name:
        dataset = python_dataset

    def preprocess_dataset(dataset):
        questions = [q.strip() for q in dataset["questions"]]
        contexts = [c.strip() for c in dataset[code_type]]
        answers = [c.strip() for c in dataset["answers"]]

        inputs = tokenizer(
            questions,
            contexts,
            text_target=answers,
            max_length=max_length,
            truncation=True
        )

        return inputs

    tokenized_dataset = dataset.map(
        preprocess_dataset,
        batched=True,
        remove_columns=dataset.column_names,
    )

    tf_dataset = core_qa_utils.prepare_tf_dataset(
        model=model,
        hf_dataset=tokenized_dataset,
        collator=data_collator,
        batch_size=batch_size,
    )
    
    loaded_weights_model = generative_qa_utils.load_weights_into_model(
        model=model, 
        model_name=model_name
    )

    questions_and_answers_df = pd.DataFrame()
    predictions_list = []
    labels_list = []
    question_contexts_list = []

    for batch, labels in tf_dataset:
        predictions = generative_qa_utils.generate_predictions(model, batch, max_length)
        decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = labels
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_predictions = [prediction.strip() for prediction in decoded_predictions]
        decoded_labels = [label.strip() for label in decoded_labels]
        predictions_list.extend(decoded_predictions)
        labels_list.extend(decoded_labels)
        question_contexts_list.extend(tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=True))

        data = {
            'question_contexts': tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=True),
            'labels': decoded_labels,
            'predictions': decoded_predictions
        }
        questions_and_answers_df = pd.concat([questions_and_answers_df, pd.DataFrame(data)], ignore_index=True)

    prediction_dataframe_list.append(questions_and_answers_df)
    model_evaluation_dir = generative_qa_paths.model_evaluation_dir / model_name
    questions_and_answers_df.to_csv(
        model_evaluation_dir / "predictions_for_manual_check.csv", index=False
    )
    i = i + 1

  0%|          | 0/19 [00:00<?, ?it/s]Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForConditionalGeneration: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight']
- This IS expected if you are initializing TFBartForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


Map:   0%|          | 0/35 [00:00<?, ? examples/s]

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
 63%|██████▎   | 12/19 [00:35<00:20,  2.94s/it]Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForConditionalGeneration: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight']
- This IS expected if you are initializing TFBartForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForConditionalGeneration were initialize

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
 68%|██████▊   | 13/19 [01:07<00:36,  6.06s/it]Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForConditionalGeneration: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight']
- This IS expected if you are initializing TFBartForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForConditionalGeneration were initialize

...

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
 89%|████████▉ | 17/19 [03:13<00:36, 18.20s/it]c:\Users\Artur\.conda\envs\question_answering\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_le

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
 95%|█████████▍| 18/19 [03:30<00:18, 18.02s/it]c:\Users\Artur\.conda\envs\question_answering\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_le

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 19/19 [03:49<00:00, 12.07s/it]
